In [3]:
import argparse
import pandas as pd
import os
from tqdm import tqdm as tqdm
import urllib.request
import numpy as np
import sys

# Downloading the dataset from Fakeeditt dataset
https://github.com/entitize/Fakeddit

change "path_of_tsv_file" varialbe accoridng to the local system path. Below are my local system paths

In [4]:
path_of_tsv_file = r"D:\UB\2nd_sem\CSE_676_DL\Project\DL_work\Code\Dataset\all_train.tsv"
df = pd.read_csv(path_of_tsv_file, sep="\t")
df = df.replace(np.nan, '', regex=True)
df.fillna('', inplace=True)

In [5]:
df.shape

(878218, 20)

## This dataset has Title of the Image, and the URL of the image, some of the URLs are expired so we need to handle that when downloading them.

In [6]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,author,clean_title,created_utc,domain,hasImage,id,image_url,linked_submission_id,num_comments,score,subreddit,title,upvote_ratio,2_way_label,3_way_label,6_way_label
0,0,0,,,Alexithymia,my walgreens offbrand mucinex was engraved wit...,1.551641e+09,i.imgur.com,True,awxhir,https://external-preview.redd.it/WylDbZrnbvZdB...,,2.0,12,mildlyinteresting,My Walgreens offbrand Mucinex was engraved wit...,0.84,1,0,0
1,1,1,155885.0,714550.0,RickSisco,,1.443822e+09,,True,cvm5uy4,http://i.imgur.com/yxrkYT8.jpg,3n7fld,,5,psbattle_artwork,,,0,2,4
2,2,2,,,VIDCAs17,this concerned sink with a tiny hat,1.534727e+09,i.redd.it,True,98pbid,https://preview.redd.it/wsfx0gp0f5h11.jpg?widt...,,2.0,119,pareidolia,This concerned sink with a tiny hat,0.99,0,2,2
3,3,3,,,prometheus1123,hackers leak emails from uae ambassador to us,1.496511e+09,aljazeera.com,True,6f2cy5,https://external-preview.redd.it/6fNhdbc6K1vFA...,,1.0,44,neutralnews,Hackers leak emails from UAE ambassador to US,0.92,1,0,0
4,4,4,282323.0,1228398.0,,,1.378792e+09,,True,cc5cbon,http://i.imgur.com/M8KTWMx.jpg,1lz1q0,,3,psbattle_artwork,,,0,2,4


# Let's try to download 15K images from the given dataset.

In [9]:
no_of_images_to_download = 15000
sample_data_df = df.sample(n=no_of_images_to_download, random_state=1)

In [11]:
sample_data_df.shape

(15000, 20)

In [15]:
clearned_data_df = pd.DataFrame()

In [16]:
sample_data_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,author,clean_title,created_utc,domain,hasImage,id,image_url,linked_submission_id,num_comments,score,subreddit,title,upvote_ratio,2_way_label,3_way_label,6_way_label
74112,74112,74112,,,,all the bearings are moving in a straight line,1.554669e+09,reddit.com,False,bakt88,,,0.0,4,confusing_perspective,All the bearings are moving in a straight line!!!,0.83,0,2,2
572633,572633,572633,,,normal_wasabi,i was able to borrow super smash bros for two ...,1.558303e+09,i.redd.it,True,bqmnjz,https://preview.redd.it/m0smpvvap8z21.jpg?widt...,,6.0,41,mildlyinteresting,I was able to borrow Super smash bros for two ...,0.86,1,0,0
154916,154916,154916,,,debensam,corners lowlife,1.564008e+09,i.redd.it,True,chf6dr,https://preview.redd.it/xfaq4xrhwbc31.png?widt...,,0.0,12,fakealbumcovers,corners - lowlife (1997),0.73,0,2,1
716664,716664,716664,255411.0,1119369.0,kouriichi,you think youre so awesome in your nonexploded...,1.394479e+09,,True,cfz6agn,http://i.imgur.com/CnfS9wT.jpg,200x20,,29,psbattle_artwork,You think youre so awesome in your non-explode...,,0,2,4
303373,303373,303373,14880.0,71867.0,Maymayfish,,1.557609e+09,,True,en5iboy,https://i.imgur.com/Wmu94rp.jpg,bnchjm,,46,psbattle_artwork,,,0,2,4


# Below cell will iterate over over datafram and download the valid image from the URL and save them as .JPG in our local system.

## We need to change "dest_img_path" accoridng to the local system on where this is being run. Please change this path when running.

In [24]:
dest_img_path = r"D:\UB\2nd_sem\CSE_676_DL\Project\DL_work\Code\Images"

pbar = tqdm(total=len(sample_data_df))

if not os.path.exists(dest_img_path):
    os.makedirs(dest_img_path)
for index, row in sample_data_df.iterrows():
    if row["hasImage"] == True and row["image_url"] != "" and row["image_url"] != "nan":
        image_url = row["image_url"]
        try:
            img_path = os.path.join(dest_img_path, row["id"] + ".jpg")
            urllib.request.urlretrieve(
                image_url, img_path)
            clearned_data_df = clearned_data_df.append(sample_data_df.iloc[index])
        except:
            pass
    pbar.update(1)
print("done")


100%|█████████████████████████████████████| 15000/15000 [34:49<00:00,  9.88it/s]

done


In [25]:
clearned_data_df.shape

(10541, 21)

In [27]:
clearned_data_df.keys()

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'author', 'clean_title', 'created_utc', 'domain',
       'hasImage', 'id', 'image_url', 'linked_submission_id', 'num_comments',
       'score', 'subreddit', 'title', 'upvote_ratio', '2_way_label',
       '3_way_label', '6_way_label'],
      dtype='object')

100%|█████████████████████████████████████| 15000/15000 [35:00<00:00,  9.88it/s]

# Let's try to understand the data, by looking at what the unique values are in those columns

In [28]:
cols = ["domain", "hasImage", "num_comments", "score", "subreddit", "upvote_ratio", "2_way_label", "3_way_label", "6_way_label"]


for col in cols:
    print("COL-",col)
    print(clearned_data_df[col].unique())
    print("*"*100)


COL- domain
['i.redd.it' nan 'theguardian.com' 'i.imgur.com' 'imgur.com'
 'web.archive.org' 'thehill.com' 'nextshark.com' 'cdn.androidbeat.com'
 'msn.com' 'archive.is' 'v.redd.it' 'miamiherald.com' 'independent.co.uk'
 'theonion.com' 'i.reddituploads.com' 'channelnewsasia.com'
 'news.cgtn.com' 'smh.com.au' 'finance.yahoo.com' 'self.satire'
 'apnews.com' 'twitter.com' 'socdn.smtown.com' 'bbc.com' 'popsci.com'
 'cnet.com' 'pbs.twimg.com' 'tampabay.com' 'fox13now.com'
 'nydailynews.com' 'tbnewswatch.com' 'abcnews.go.com' 'jsonline.com'
 'nj.com' 'gfycat.com' 'theargus.co.uk' 'axios.com' 'reuters.com'
 'eddietrombone.com' 'youtube.com' 'bbc.co.uk' 'cbc.ca'
 'opposingviews.com' 'dailytelegraph.com.au' 'ppcdn.500px.org'
 'ibtimes.com' 'gothamist.com' 'money.cnn.com' 'pics.me.me'
 'edmontonjournal.com' 'theglobeandmail.com' 'actionnewsjax.com'
 'deseret.com' 'clickhole.com' 'latimes.com' 'onionstudios.com'
 'reason.com' 'bloomberg.com' 'economist.com' 'nesn.com' 'usatoday.com'
 'magnum-mania.

In [28]:
cleaned_simple_df = pd.DataFrame()

In [29]:
cleaned_simple_df["clean_title"] = clearned_data_df["clean_title"]
cleaned_simple_df["image_url"] = clearned_data_df["image_url"]
cleaned_simple_df["title"] = clearned_data_df["title"]
cleaned_simple_df["2_way_label"] = clearned_data_df["2_way_label"]
cleaned_simple_df["3_way_label"] = clearned_data_df["3_way_label"]
cleaned_simple_df["6_way_label"] = clearned_data_df["6_way_label"]
cleaned_simple_df["id"] = clearned_data_df["id"]
cleaned_simple_df["num_comments"] = clearned_data_df["num_comments"]

In [30]:
cleaned_simple_df.shape

(10541, 8)

In [31]:
cleaned_simple_df.to_csv("cleaned_simple.csv")